<a href="https://colab.research.google.com/github/sidhu2690/GEANT4/blob/main/HGCAL_Layer_Prototype_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import numpy as np
import pandas as pd
from scipy.stats import moyal
from scipy.optimize import curve_fit

def landau_pdf(x, mpv, sigma, amplitude):
    loc = mpv - sigma * np.log(2)
    scale = sigma
    return amplitude * moyal.pdf(x, loc=loc, scale=scale)

def fit_landau_and_cut(edep_values):
    counts, bins = np.histogram(edep_values, bins=50)
    if np.max(counts) == 0:
        return None, None

    mpv_guess = np.percentile(edep_values, 75)
    sigma_guess = np.std(edep_values) * 0.5
    amplitude_guess = np.max(counts)
    bin_centers = (bins[:-1] + bins[1:]) / 2

    try:
        popt, _ = curve_fit(landau_pdf, bin_centers, counts,
                            p0=[mpv_guess, sigma_guess, amplitude_guess],
                            maxfev=10000, bounds=([0, 0, 0], [np.inf, np.inf, np.inf]))
        mpv = popt[0]
        cut_value = 0.75 * mpv
        return mpv, cut_value
    except:
        return None, None

def process_cellwise(input_file, output_file='processed_hits.csv'):
    df = pd.read_csv(input_file, sep=r'\s+')
    df_filtered = df[(df['Layer'] > 0) & (df['Edep_MeV'] > 0)].copy()

    cellSize = 50

    df_filtered['i'] = np.round(df_filtered['X_mm'] / cellSize)
    df_filtered['j'] = np.round(df_filtered['Y_mm'] / cellSize)
    df_filtered['xi'] = df_filtered['i'] * cellSize
    df_filtered['yi'] = df_filtered['j'] * cellSize
    df_filtered['zi'] = df_filtered['Z_mm']

    processed_hits = []

    grouped = df_filtered.groupby(['Layer', 'i', 'j'])

    for (layer, i, j), group in grouped:
        edep_values = group['Edep_MeV'].values

        mpv, cut_value = fit_landau_and_cut(edep_values)

        if mpv is None:
            continue

        hits_above_cut = group[group['Edep_MeV'] >= cut_value]

        for _, row in hits_above_cut.iterrows():
            xi, yi, zi = row['xi'], row['yi'], row['zi']
            edep = row['Edep_MeV']

            ri = np.sqrt(xi**2 + yi**2 + zi**2)
            phi_rad = np.arctan2(yi, xi)
            phi_deg = np.degrees(phi_rad)
            theta_rad = np.arccos(zi / ri) if ri > 0 else 0
            theta_deg = np.degrees(theta_rad)
            eta = -np.log(np.tan(theta_rad / 2.0)) if 0 < theta_rad < np.pi else 0.0

            processed_hits.append({
                'layer': int(layer), 'i': int(i), 'j': int(j),
                'xi': round(xi, 3), 'yi': round(yi, 3), 'zi': round(zi, 3),
                'theta': round(theta_deg, 2), 'phi': round(phi_deg, 2),
                'eta': round(eta, 2), 'edep': round(edep, 6)
            })

    df_output = pd.DataFrame(processed_hits)
    df_output.to_csv(output_file, index=False)

    print(f"Number of Entries: {len(df_output)}")
    print(f"\n{df_output.head(10)}")

    return df_output

df_output = process_cellwise("/content/particle_tracking.txt")

Number of Entries: 587

   layer   i   j      xi      yi      zi  theta     phi   eta      edep
0      1 -21  12 -1050.0   600.0  3221.4  20.58  150.26  1.71  0.086918
1      1  -8   6  -400.0   300.0  3221.4   8.82  143.13  2.56  0.207414
2      1  -6   7  -300.0   350.0  3221.4   8.14  130.60  2.64  0.247754
3      1  -3  26  -150.0  1300.0  3221.4  22.11   96.58  1.63  0.085882
4      1   0  25     0.0  1250.0  3221.4  21.21   90.00  1.68  0.094520
5      1   1  24    50.0  1200.0  3221.4  20.45   87.61  1.71  0.132449
6      1   3   7   150.0   350.0  3221.4   6.74   66.80  2.83  0.127865
7      1   4 -17   200.0  -850.0  3221.4  15.17  -76.76  2.02  0.088121
8      1   4  16   200.0   800.0  3221.4  14.36   75.96  2.07  0.090283
9      1   5  -5   250.0  -250.0  3221.4   6.26  -45.00  2.91  0.098834
